In [1]:
from keybert import KeyBERT

In [2]:

doc = """
        I knew that my girlfriend was depressed for a long time and actively recommended me to visit the hospital. 
        I visited the hospital for the first time last week and scored more than 30 points on the depression scale test. 
        Currently, he is under medication, and the hospital recommends counseling treatment at the same time, so he is also considering this part.

        In this situation, I'm so worried about how I should support this person.
        My girlfriend is a job seeker who graduated from a top university (I mean I've managed my life well on my own) and her cycle is not fixed.

        I'm worried that there's no fixed cycle for a day. 
        Being lethargic at home all day and how to deal with a girlfriend who feels ashamed that she has done nothing herself in that lethargy... 
        I want you to look at my actual case and answer each one. 

        As a boyfriend, I am an office worker living with my family. 
        I am also aware of the lack of time limitations even though I cannot always be with you and try to do something together.

        1. I don't know what I should tell my girlfriend to get out of bed and rest well, who feels ashamed of herself without doing anything because she is lazy. 
        I think I should do this, too, as I say every time, "Would you like to wash up and think about it?" in a gentle and soothing way. 
        How am I supposed to deal with this girlfriend's sense of shame and how am I supposed to treat this person?

        2. When my girlfriend, who is a job seeker but still has insufficient specifications, spent a proud day doing small things such as organizing her room, nail clippers, etc., 
        I'm a person who says, "Why don't you go outside the house and prepare for a certificate?" Should I compliment you on small things? 
        My girlfriend knows that I'm told to go out, but I think it's good for depression when I go out and work and have a regular life.

        In the case of No. 2, even if I say it in a good way, the good atmosphere suddenly subsides, so I'm determined to say it once, but if I say it, I can't undo the down feeling. 
        And I always worry about what if my girlfriend, who always lives alone, makes an extreme decision. I can't be with you all the time.

        I'd like psychiatrists to tell me how to treat someone with depression. I really like and love this person and I'm really rooting for him. I hope you can help me.
      """


In [3]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
movies = pd.read_csv('data/IMDB-Movie-Data.csv')
movies

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0


In [5]:
movies = movies[['Title','Director','Description']]
movies

,Title,Director,Description
0,Guardians of the Galaxy,James Gunn,A group of intergalactic criminals are forced ...
1,Prometheus,Ridley Scott,"Following clues to the origin of mankind, a te..."
2,Split,M. Night Shyamalan,Three girls are kidnapped by a man with a diag...
3,Sing,Christophe Lourdelet,"In a city of humanoid animals, a hustling thea..."
4,Suicide Squad,David Ayer,A secret government agency recruits some of th...
...,...,...,...
995,Secret in Their Eyes,Billy Ray,"A tight-knit team of rising investigators, alo..."
996,Hostel: Part II,Eli Roth,Three American college students studying abroa...
997,Step Up 2: The Streets,Jon M. Chu,Romantic sparks occur between two dance studen...
998,Search Party,Scot Armstrong,A pair of friends embark on a mission to reuni...


In [6]:
movies2 = pd.read_csv('data/tmdb_5000_movies.csv')

In [7]:
movies2

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [8]:
movies2.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [9]:
import numpy as np
movies2 = movies2[['title','overview']]
movies2.columns = ['Title','Description']
movies2['Director'] = np.nan
movies2 = movies2[['Title','Director','Description']]
movies2

C:\Users\pc\AppData\Local\Temp\ipykernel_50868\1479077215.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies2['Director'] = np.nan


,Title,Director,Description
0,Avatar,NaN,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,NaN,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,NaN,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,NaN,Following the death of District Attorney Harve...
4,John Carter,NaN,"John Carter is a war-weary, former military ca..."
...,...,...,...
4798,El Mariachi,NaN,El Mariachi just wants to play his guitar and ...
4799,Newlyweds,NaN,A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered",NaN,"""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,NaN,When ambitious New York attorney Sam is sent t...


In [10]:
movies = pd.concat([movies,movies2]).reset_index(drop=True)
movies

,Title,Director,Description
0,Guardians of the Galaxy,James Gunn,A group of intergalactic criminals are forced ...
1,Prometheus,Ridley Scott,"Following clues to the origin of mankind, a te..."
2,Split,M. Night Shyamalan,Three girls are kidnapped by a man with a diag...
3,Sing,Christophe Lourdelet,"In a city of humanoid animals, a hustling thea..."
4,Suicide Squad,David Ayer,A secret government agency recruits some of th...
...,...,...,...
5798,El Mariachi,NaN,El Mariachi just wants to play his guitar and ...
5799,Newlyweds,NaN,A newlywed couple's honeymoon is upended by th...
5800,"Signed, Sealed, Delivered",NaN,"""Signed, Sealed, Delivered"" introduces a dedic..."
5801,Shanghai Calling,NaN,When ambitious New York attorney Sam is sent t...


In [11]:
movies = movies.drop_duplicates(subset=['Title','Description']).reset_index(drop=True)
movies

,Title,Director,Description
0,Guardians of the Galaxy,James Gunn,A group of intergalactic criminals are forced ...
1,Prometheus,Ridley Scott,"Following clues to the origin of mankind, a te..."
2,Split,M. Night Shyamalan,Three girls are kidnapped by a man with a diag...
3,Sing,Christophe Lourdelet,"In a city of humanoid animals, a hustling thea..."
4,Suicide Squad,David Ayer,A secret government agency recruits some of th...
...,...,...,...
5724,El Mariachi,NaN,El Mariachi just wants to play his guitar and ...
5725,Newlyweds,NaN,A newlywed couple's honeymoon is upended by th...
5726,"Signed, Sealed, Delivered",NaN,"""Signed, Sealed, Delivered"" introduces a dedic..."
5727,Shanghai Calling,NaN,When ambitious New York attorney Sam is sent t...


In [12]:
movies.to_csv('data/Movie_data.csv',index=False)

In [13]:
kw_model = KeyBERT()

def preprocess_text(text):

    key1 = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words="english",top_n=5)
    key2 = kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 2), stop_words="english",use_mmr=True, diversity=0.7)
    key3 = kw_model.extract_keywords(text, keyphrase_ngram_range=(3, 3), stop_words="english",use_maxsum=True, nr_candidates=20)

    key_list = []
    for k1 in key1:
        key_list.append(k1[0])

    for k2 in key2:
        key_list.append(k2[0])

    for k3 in key3:
        key_list.append(k3[0])

    preprocessed_text = ' '.join(key_list)

    return preprocessed_text

In [14]:
#import nltk
#nltk.download("popular")

In [15]:
sentences = []

for overview in movies['Description']:
    try:
        sentences.append(word_tokenize(preprocess_text(overview)))
    except:
        pass


In [16]:
model = Word2Vec(sentences, vector_size=300, window=5, min_count=5, workers=4, sg=1)

In [17]:
model.wv.save_word2vec_format('model/movie_w2v') # 모델 저장

In [18]:
model = KeyedVectors.load_word2vec_format("model/movie_w2v")

In [19]:
def get_vector(text, model):
    vectors = []
    for word in word_tokenize(text):
        if word in model.key_to_index:
            vectors.append(model[word])
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)


In [33]:
def recommend_movie(text, top_n=5):
    # 입력된 텍스트를 전처리합니다.
    preprocessed_text = preprocess_text(text)

    # 입력된 텍스트의 벡터를 구합니다.
    text_vector = get_vector(preprocessed_text, model)

    # 각 영화 데이터의 벡터를 구합니다.
    movie_vectors = []
    movie_title = []
    movie_director = []

    for i,overview in enumerate(movies['Description']):
        try:
            movie_vectors.append(get_vector(preprocess_text(overview), model))
            movie_title.append(movies['Title'][i])
            movie_director.append(movies['Director'][i])
        except:
            pass

    # 입력된 텍스트와 각 영화 데이터의 유사도를 측정합니다.
    similarities = [cosine_similarity(text_vector.reshape(1, -1), movie_vector.reshape(1, -1)).flatten()[0]
                    for movie_vector in movie_vectors]

    # 유사도가 가장 높은 상위 n개의 영화를 추천합니다.
    top_n_movies = pd.DataFrame({'Title': movie_title, 'Director': movie_director ,'similarity': similarities}) \
        .sort_values(by='similarity', ascending=False).head(top_n).reset_index(drop=True)
    
    

    return top_n_movies

In [34]:
rm = recommend_movie(doc)
rm

,Title,Director,similarity
3088,One Missed Call,One Missed Call,0.993059
5222,Home Run,Home Run,0.992224
839,Horrible Bosses 2,Horrible Bosses 2,0.992071
479,Forgetting Sarah Marshall,Forgetting Sarah Marshall,0.991721
1136,Madagascar 3: Europe's Most Wanted,Madagascar 3: Europe's Most Wanted,0.991714


In [35]:
rm = rm.reset_index(drop=True)

In [39]:
import json

recomend = dict()
recomend_movie = []

for i,_ in enumerate(rm):
    recomend_movie.append({'title':rm['Title'][i],'producer':rm['Director'][i]})


In [40]:
recomend_movie

[{'title': 'One Missed Call', 'producer': 'One Missed Call'},
 {'title': 'Home Run', 'producer': 'Home Run'},
 {'title': 'Horrible Bosses 2', 'producer': 'Horrible Bosses 2'}]

In [46]:
recomend = {'movie':recomend_movie}
recomend

{'movie': [{'title': 'One Missed Call', 'producer': 'One Missed Call'},
  {'title': 'Home Run', 'producer': 'Home Run'},
  {'title': 'Horrible Bosses 2', 'producer': 'Horrible Bosses 2'}]}

In [47]:
file_path = "./sample.json"
with open(file_path, 'w') as outfile:
    json.dump(recomend, outfile, indent=4)